In [1]:
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import nltk
from gensim.models import Word2Vec
from nltk.corpus import brown, movie_reviews, treebank
import numpy as np
import gensim

In [122]:
# Train with Brown corpus
tr_model = Word2Vec(brown.sents())

In [2]:
# Train with Google model
ggle_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
# This removes all words not in the Google model
def nonExistWordRemoval( wordList ):
    return [w for w in wordList if w in ggle_model.vocab ]

# From parsing team: This removes all the stop words
def stopWordRemoval( wordList ):
    swords = nltk.corpus.stopwords.words('english')
    return [w for w in wordList if w not in swords ]

In [30]:
# From parsing team: formats the questions
data = []
answers = []
with open('AI2-8thGr-NDMC-Feb2016-Train.jsonl') as f:
    for line in f:
        question=json.loads(line)
        q_dict={}
        temp_str=question['question']['stem'].encode('utf-8').strip()
        q_dict['1Q']=re.sub("[^\w]", " ",  temp_str).split()
        q_dict['1Q'] = stopWordRemoval(q_dict['1Q'])
        q_dict['1Q'] = nonExistWordRemoval(q_dict['1Q'])
        for opt in question['question']['choices']:
            temp_str=opt['text'].encode('utf-8').strip()
            q_dict[opt['label'].encode('utf-8').strip()]=re.sub("[^\w]", " ",  temp_str).split()
            q_dict[opt['label'].encode('utf-8').strip()] = nonExistWordRemoval(q_dict[opt['label'].encode('utf-8').strip()])
        data.append(q_dict)

# Formats the answers
with open('AI2-8thGr-NDMC-Feb2016-Train.jsonl') as f:
    for line in f:
        question=json.loads(line)
        q_dict={}
        temp_str=question['answerKey'].encode('utf-8').strip()
        q_dict['A']=re.sub("[^\w]", " ",  temp_str).split()
        q_dict['A'] = stopWordRemoval(q_dict['A'])
        answers.append(q_dict)

In [51]:
# Function that gets the best answer of the question
# Input question will be an array whose first element is the question and rest are answers
# I use "getVal" function below to call getScore/getScoreG functions
# getScoreG uses Google library and uses similarity calculation, gets around 34% accuracy 
# getScore uses Brown library and uses mv calculation, gets around 25% accuracy

# This is a dummy value for failed attemps
ups = 0

def getScoreG(questionAnswers):
    # Creating empty answers and its scores
    answers = questionAnswers[1:]
    aScore = np.zeros([100,len(questionAnswers)-1])
    
    # Get the question score
    q = questionAnswers[0]
    
    #I think we can do it as matrix but just having a loop for now
    result = np.zeros([1,len(questionAnswers)-1])
    temp = np.zeros([1,len(questionAnswers)-1])
    for i in range(1,len(questionAnswers)):
        answers[i-1] = questionAnswers[i]
        tempa = questionAnswers[i]
        try:
            result[:,i-1]=ggle_model.n_similarity(tempa,q)
            temp[:,i-1]=ggle_model.n_similarity(tempa,q)
        except:
            result[:,i-1] = ups
            temp[:,i-1]=ups

    # Get the top 2 index of the answers (Min distance from question)
    resultTopInd = np.argmax(result)
    resultTopInd.astype('str')
    # Set the top answer to infinity to get the second best answer
    temp[:,resultTopInd] = float('inf')
    resultSecInd = np.argmax(temp)
    if (resultTopInd==0):
        resultTopInd = "A"
    elif (resultTopInd==1):
        resultTopInd = "B"
    elif (resultTopInd==2):
        resultTopInd = "C"
    elif (resultTopInd==3):
        resultTopInd = "D"
    
    return resultTopInd

In [120]:
# Function that gets the best answer of the question
# Input question will be an array whose first element is the question and rest are answers
# I use "getVal" function below to call getScore/getScoreG functions
# getScoreG uses Google library and uses similarity calculation, gets around 34% accuracy 
# getScore uses Brown library and uses mv calculation, gets around 25% accuracy

# This is a dummy value for failed attemps
ups = np.zeros(100)
ups[:] = 0

def getScore(questionAnswers):
    # Creating empty answers and its scores
    answers = questionAnswers[1:]
    aScore = np.zeros([100,len(questionAnswers)-1])
    
    # Get the question score
    q = questionAnswers[0]
    try:
        qScore = np.mean(tr_model.wv[q],axis=0)
    except:
        qScore = ups
    
    #I think we can do it as matrix but just having a loop for now
    result = np.zeros([1,len(questionAnswers)-1])
    temp = np.zeros([1,len(questionAnswers)-1])
    for i in range(1,len(questionAnswers)):
        answers[i-1] = questionAnswers[i]
        tempa = questionAnswers[i]
        scorei = np.zeros([100,len(answers[i-1])])
        for j in range(0,len(answers[i-1])):
            try: 
                scorei[:,j] = tr_model.wv[tempa[j]]
            except:
                scorei[:,j] = ups
        aScore[:,i-1] = np.mean(scorei,axis=1)
        result[:,i-1]=np.linalg.norm(qScore-aScore[:,i-1])
        #result[:,i-1] = np.mean(qScore-aScore[:,i-1])
        #temp[:,i-1] = np.mean(qScore-aScore[:,i-1])
        temp[:,i-1]=np.linalg.norm(qScore-aScore[:,i-1])

    # Get the top 2 index of the answers (Min distance from question)
    resultTopInd = np.argmin(result)
    resultTopInd.astype('str')
    # Set the top answer to infinity to get the second best answer
    temp[:,resultTopInd] = float('inf')
    resultSecInd = np.argmin(temp)
    if (resultTopInd==0):
        resultTopInd = "A"
    elif (resultTopInd==1):
        resultTopInd = "B"
    elif (resultTopInd==2):
        resultTopInd = "C"
    elif (resultTopInd==3):
        resultTopInd = "D"
    
    # Get the score difference of the top 2 answers
    #diff = np.abs(result[resultTopInd]-result[resultSecInd])
    
    # If the difference is smaller than the set threshold, we go to KB
    # Not sure how to write that yet, so breaking it
    #if diff<1:
        #break
    # If the difference is big enough, we return the index of the answer
    #else:
    return resultTopInd,result

In [123]:
# This is for testing
question = ['move', 'flight', 'tire']
a2 = ['test','it']
a3 = ['car', 'plane', 'boat']
a4 = ['move','tire']
test = [question, a2,a3,a4]
getScore(test)

('C', array([[ 4.21871766,  1.61654222,  0.4732698 ]]))

In [119]:
# getVal function takes a formated dictionary and calls getScore or getScoreG to calculate the answers to the questions
def getVal(dataDict):
    ans = np.zeros(len(dataDict))
    ans = ans.astype('str')
    count = 0
    #for i in range(0,len(dataDict)):
    for i in range(0,293):
        tempL = dataDict[i].values()
        tags = ['1Q','A','B','C','D']
        for j in range (0,5):
            curTag = tags[j]
            #tempL[j]=dataDict[i].values()[j]
            try:
                tempL[j]=dataDict[i][curTag]
            except:
                a=1
        ans[i] = getScoreG(tempL)
        print(i)
        #print(tempL,ans[i],answers[i].values()[0][0])
        #print(dataDict[i].values(),tempL,answers[i].values()[0][0])
        if ans[i] == answers[i].values()[0][0]:
            count = count+1
    print(len(dataDict),count)
        
        
getVal(data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27